In [1]:
import xarray as xr
import pandas as pd
import plotly.express as px

In [2]:
hot_ctd = xr.open_zarr("../../2_processed/hot_ctd.zarr")
hot_primary_production = xr.open_zarr("../../2_processed/hot_primary_production.zarr")
hot_zooplankton = xr.open_zarr("../../2_processed/hot_zooplankton.zarr")

In [3]:
hot_zooplankton

<xarray.Dataset> Size: 2MB
Dimensions:  (time: 548, is_day: 2, lat: 1, lon: 1, depth: 8, frac: 5)
Coordinates:
  * depth    (depth) int64 64B 50 100 150 200 250 300 350 400
  * frac     (frac) float64 40B 0.2 0.5 1.0 2.0 5.0
  * is_day   (is_day) bool 2B False True
  * lat      (lat) float64 8B 22.75
  * lon      (lon) int64 8B -158
  * time     (time) datetime64[ns] 4kB 1994-02-17 1994-02-18 ... 2022-09-02
Data variables:
    abnd     (time, is_day, lat, lon, depth, frac) float64 351kB dask.array<chunksize=(274, 2, 1, 1, 8, 5), meta=np.ndarray>
    carb     (time, is_day, lat, lon, depth, frac) float64 351kB dask.array<chunksize=(274, 2, 1, 1, 8, 5), meta=np.ndarray>
    dwt      (time, is_day, lat, lon, depth, frac) float64 351kB dask.array<chunksize=(274, 2, 1, 1, 8, 5), meta=np.ndarray>
    nit      (time, is_day, lat, lon, depth, frac) float64 351kB dask.array<chunksize=(274, 2, 1, 1, 8, 5), meta=np.ndarray>
    svol     (time, is_day, lat, lon, depth, frac) float64 351kB dask.array<chunksize=(274, 2, 1, 1, 8, 5), meta=np.ndarray>
    vol      (time, is_day, lat, lon, depth, frac) float64 351kB dask.array<chunksize=(274, 2, 1, 1, 8, 5), meta=np.ndarray>
    wwt      (time, is_day, lat, lon, depth, frac) float64 351kB dask.array<chunksize=(274, 2, 1, 1, 8, 5), meta=np.ndarray>

## Manage index

---


In [4]:
times = pd.DataFrame(
    {
        "btl_time": hot_ctd.time.to_series(),
        "zpk_time": hot_zooplankton.time.to_series(),
        "pp_time": hot_primary_production.time.to_series(),
    }
)
fig = px.histogram(
    times,
    x=["btl_time", "zpk_time", "pp_time"],
    title="time distribution",
    opacity=0.5,
    log_y=True,
    nbins=30,
    barmode="overlay",
    marginal="box",
    labels={"value": "time (m)", "variable": "Datasets"},
)

fig.show()

In [5]:
depths = pd.DataFrame(
    {
        "ctd_depth": hot_ctd.where(hot_ctd.depth < 400, drop=True).depth.to_series(),
        "zpk_depth": hot_zooplankton.depth.to_series(),
        "pp_depth": hot_primary_production.depth.to_series(),
    }
)
px.box(
    depths.stack().reset_index().rename(columns={"level_1": "dataset"}),
    x="dataset",
    y="depth",
)

## Gathering

---


In [6]:
hot_zooplankton = hot_zooplankton.rename({"nit": "nitrogen"})
hot_ctd = hot_ctd.rename({"nit": "nitrate"})

In [7]:
final_dataset = xr.merge([hot_zooplankton, hot_primary_production, hot_ctd])
final_dataset.load()

<xarray.Dataset> Size: 4MB
Dimensions:   (depth: 8, frac: 5, is_day: 2, lat: 1, lon: 1, time: 687)
Coordinates:
  * depth     (depth) int64 64B 50 100 150 200 250 300 350 400
  * frac      (frac) float64 40B 0.2 0.5 1.0 2.0 5.0
  * is_day    (is_day) bool 2B False True
  * lat       (lat) float64 8B 22.75
  * lon       (lon) int64 8B -158
  * time      (time) datetime64[ns] 5kB 1988-10-31 1988-12-02 ... 2022-09-02
Data variables: (12/25)
    abnd      (time, is_day, lat, lon, depth, frac) float64 440kB nan ... nan
    carb      (time, is_day, lat, lon, depth, frac) float64 440kB nan ... nan
    dwt       (time, is_day, lat, lon, depth, frac) float64 440kB nan ... nan
    nitrogen  (time, is_day, lat, lon, depth, frac) float64 440kB nan ... nan
    svol      (time, is_day, lat, lon, depth, frac) float64 440kB nan ... nan
    vol       (time, is_day, lat, lon, depth, frac) float64 440kB nan ... nan
    ...        ...
    press     (time, lat, lon, depth) float64 44kB 25.0 76.0 126.0 ... nan nan
    sal       (time, lat, lon, depth) float64 44kB 35.21 35.22 35.19 ... nan nan
    sigma     (time, lat, lon, depth) float64 44kB 23.13 23.85 24.63 ... nan nan
    temp      (time, lat, lon, depth) float64 44kB 26.21 23.86 21.07 ... nan nan
    theta     (time, lat, lon, depth) float64 44kB 26.21 23.84 21.04 ... nan nan
    xmiss     (time, lat, lon, depth) float64 44kB 3.42 0.4096 ... nan nan

In [8]:
final_dataset.to_zarr("../../3_post_processed/hot_product.zarr", mode="w")